In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
from keras.layers import RepeatVector
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

## visualize model
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


## Data Set

In [2]:
in_seq1 = np.arange(10, 100, 10)
in_seq2 = np.arange(15, 105, 10)
out_seq = in_seq1 + in_seq2
print(out_seq)
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[ 25  45  65  85 105 125 145 165 185]
[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [3]:
#
n_step = 3
pred_step = 1
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step-1)):
        x_list.append(series[s:s+n_step, :2]),
        y_list.append(series[s+n_step-1, -1])
    X, y = np.array(x_list), np.array(y_list)
    return X, y
#
X, y = split_sequence(dataset, n_step, pred_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


## Vanilla LSTM

- We will use a Vanilla LSTM where 
  - the number of time steps and 
  - parallel series (features) are specified for the input layer via the input shape argument
- When making a prediction, the model expects three time steps for two input time series.
- We can predict the next value in the output series providing the input values of:

In [4]:
X.shape

(7, 3, 2)

In [5]:
n_steps = 3
n_features = 2
#
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
#
model.fit(X, y, epochs=200, verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [6]:
x_test = np.array([[80, 85], [90, 95], [100, 105]])
x_test_lstm = x_test.reshape(1, n_steps, n_features)
yhat = model.predict(x_test_lstm)
print(yhat)

[[207.48567]]


## 9.3.2 Multiple Parallel Series

In [7]:
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [8]:
#
n_steps = 3
pred_step = 1
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step-1)):
        x_list.append(series[s:s+n_step]),
        y_list.append(series[s+n_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y
#
X_para, y_para = split_sequence(dataset, n_steps, pred_step)
for i in range(X_para.shape[0]):
    print(X_para[i], y_para[i])

[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [9]:
print(X_para.shape, y_para.shape)

(6, 3, 3) (6, 3)


In [10]:
# define model
n_features = X_para.shape[2]
print(n_steps, n_features)
#
model_para = Sequential()
model_para.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model_para.add(LSTM(100, activation='relu'))
model_para.add(Dense(n_features))
model_para.compile(optimizer='adam', loss='mse')
model_para.fit(X_para, y_para, epochs=200, verbose=0)

3 3


In [13]:
x_test_para = np.array([[70, 75, 145], [80, 85, 165], [90, 95, 185]])
print(x_test_para.shape)
x_test_para_lstm = x_test_para.reshape(1, n_steps, n_features)
print(x_test_para_lstm)

(3, 3)
[[[ 70  75 145]
  [ 80  85 165]
  [ 90  95 185]]]


In [14]:
yhat_para = model_para.predict(x_test_para_lstm)
print(yhat_para)### vanilla LSTM model

[[100.50127 105.33746 203.69699]]


In [15]:
plot_model(model_para, to_file='9_3_2_lstm.png', show_shapes=True, show_layer_names=True)

### Stacked LSTM model from Parallel series
<img src="./9_3_2_lstm.png" alt="Drawing" style="width: 600px;"/>

## 9.4 Multi-step LSTM Models

In [16]:
dataset_cont = np.arange(10, 100, 10)
print(dataset_cont)

[10 20 30 40 50 60 70 80 90]


In [17]:
#
n_steps = 3
pred_step = 2
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step-1)):
        x_list.append(series[s:s+n_step]),
        y_list.append(series[s+n_step:s+n_step+pred_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y
#
X, y = split_sequence(dataset_cont, n_step, pred_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


### 9.4.2 Vector Output Model

- The LSTM expects data to have a three-dimensional structure of [samples, timesteps, features], and in this case, we only have one feature so the reshape is straightforward

In [18]:
n_features = 1
X_mstep = X.reshape(X.shape[0], n_steps, n_features)
print(X_mstep.shape)

(5, 3, 1)


In [19]:
# define model
model_mstep_vector = Sequential()
model_mstep_vector.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model_mstep_vector.add(LSTM(100, activation='relu'))
model_mstep_vector.add(Dense(pred_step))
model_mstep_vector.compile(optimizer='adam', loss='mse')
#
model_mstep_vector.fit(X_mstep, y, epochs=50, verbose=0)

In [20]:
x_test_mstep = np.array([50, 60, 70])
x_test_mstep_vector = x_test_mstep.reshape(1, n_steps, n_features)
print(x_test_mstep_vector)
yhat_mstep_vector = model_mstep_vector.predict(x_test_mstep_vector)
print(yhat_mstep_vector)

[[[50]
  [60]
  [70]]]
[[77.71273 88.14805]]


### 9.4.3 Encoder-Decoder Model

In [21]:
print(X_mstep)

[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]]


In [22]:
# define encoder model 
model_endec = Sequential()
model_endec.add(LSTM(100, activation='relu' , input_shape=(n_steps, n_features)))
# repeat encoding
model_endec.add(RepeatVector(pred_step))
# define decoder model 
model_endec.add(LSTM(100, activation='relu' , return_sequences=True))
# define model output
model_endec.add(TimeDistributed(Dense(1)))
#
model_endec.compile(optimizer='adam', loss='mse')
#

In [23]:
print(y)

[[40 50]
 [50 60]
 [60 70]
 [70 80]
 [80 90]]


In [24]:
y_mstep = y.reshape(y.shape[0], y.shape[1], n_features)
print(y_mstep)

[[[40]
  [50]]

 [[50]
  [60]]

 [[60]
  [70]]

 [[70]
  [80]]

 [[80]
  [90]]]


In [25]:
# fit model 
model_endec.fit(X_mstep, y_mstep, epochs=100, verbose=0)

In [26]:
print(x_test_mstep)

[50 60 70]


In [27]:
x_test_mstep_endec = x_test_mstep.reshape(1, n_steps, n_features)
yhat_endec = model_endec.predict(x_test_mstep_endec)
print(yhat_endec)

[[[80.068245]
  [90.14806 ]]]


## 9.5.1 Multiple Input Multi-step Output

In [28]:
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [29]:
#
n_step = 3
pred_step = 2
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step-2)):
        x_list.append(series[s:s+n_step, :2]),
        y_list.append(series[s+n_step-1:s+n_step-1+pred_step, -1])
    X, y = np.array(x_list), np.array(y_list)
    return X, y
#
X, y = split_sequence(dataset, n_step, pred_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [30]:
n_feature = X.shape[2]
print(n_feature)

2


In [31]:
# define model
model_mvms = Sequential()
model_mvms.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_step, n_feature)))
model_mvms.add(LSTM(100, activation='relu'))
model_mvms.add(Dense(pred_step))
model_mvms.compile(optimizer='adam', loss='mse')
#
model_mvms.fit(X, y, epochs=200, verbose=0)

In [32]:
x_test = np.array([[70, 75], [80, 85], [90, 95]])
print(x_test.shape)
x_test_mvms = x_test.reshape(1, n_step, n_feature)
print(x_test_mvms)

(3, 2)
[[[70 75]
  [80 85]
  [90 95]]]


In [33]:
yhat_mvms = model_mvms.predict(x_test_mvms)
print(yhat_mvms)

[[186.67769 208.21722]]


In [34]:
plot_model(model_mvms, to_file='9_5_1_lstm.png', show_shapes=True, show_layer_names=True)

### Stacked LSTM model from Multi-variate and Multi-step
<img src="./9_5_1_lstm.png" alt="Drawing" style="width: 600px;"/>

## 9.5.2 Multiple Parallel Input and Multi-step Output

In [35]:
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [37]:
#
n_step = 3
pred_step = 2
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step-1)):
        x_list.append(series[s:s+n_step]),
        y_list.append(series[s+n_step:s+n_step+pred_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y
#
X, y = split_sequence(dataset, n_step, pred_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [39]:
print(X.shape, y.shape)
n_features = X.shape[2]
n_steps = X.shape[1]

(5, 3, 3) (5, 2, 3)


In this case, we will use the Encoder-Decoder model.

In [40]:
# define encoder model 
model_endec = Sequential()
model_endec.add(LSTM(100, activation='relu' , input_shape=(n_steps, n_features)))
# repeat encoding
model_endec.add(RepeatVector(pred_step))
# define decoder model 
model_endec.add(LSTM(100, activation='relu' , return_sequences=True))
# define model output
model_endec.add(TimeDistributed(Dense(n_features)))
#
model_endec.compile(optimizer='adam', loss='mse')
#
model_endec.fit(X, y, epochs=300, verbose=0)

In [41]:
x_test = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
print(x_test.shape)

(3, 3)


In [44]:
x_test_mp = x_test.reshape(1, n_steps, n_features)
print(x_test_mp.shape)

(1, 3, 3)


In [45]:
yhat_mp = model_endec.predict(x_test_mp)
print(yhat_mp)

[[[ 90.93455   94.83655  184.76512 ]
  [101.117546 104.210594 206.26268 ]]]
